# 包的引入

In [9]:
import colorsys
from PIL import Image
import numpy as np

# 一些工具函数

## 背景处理函数


### 用于判断是像素是否为低饱和高明度、是否为低明度

如果一个像素是低饱和高明度的, 那么我们考虑这个部分是背景中的一部分, 下面的函数用于判断一个像素是否为低饱和高明度

In [10]:
def judge_low_s_high_l(r, g, b):
    # 定义RGB颜色
    r, g, b = r/255.0, g/255.0, b/255.0

    # 将RGB颜色转换为HSL颜色
    h, l, s = colorsys.rgb_to_hls(r, g, b)

    # 定义饱和度和亮度的阈值
    saturation_threshold = 0.6
    max_s = 0.85
    lightness_threshold = 0.8

    # 判断像素是否为高明度低饱和
    if (s >= max_s or s <= saturation_threshold) and l >= lightness_threshold:
        return True
    else:
        return False

如果一个像素是低明度的, 那么我们考虑这个部分是背景中的一部分, 下面的函数用于判断一个像素是否为低明度的

In [11]:
def judge_low_l(r, g, b):
    # 定义RGB颜色
    r, g, b = r/255.0, g/255.0, b/255.0

    # 将RGB颜色转换为HSL颜色
    h, l, s = colorsys.rgb_to_hls(r, g, b)

    # 定义饱和度和亮度的阈值
    lightness_threshold = 0.2

    # 判断像素是否为高明度低饱和
    if l <= lightness_threshold:
        return True
    else:
        return False

### 判断一个像素是否在白色附近

在实际应用中, 发现上面两个函数还不足以很好的去除背景, 所以使用该函数辅助进行白色背景的去除

In [12]:
def is_near_white(pixel):
    r, g, b, a = pixel

    # Define the threshold for white color
    white_threshold = 150

    # Check if the pixel is near white
    if r >= white_threshold and g >= white_threshold and b >= white_threshold:
        return True
    else:
        return False

## 描边函数

1. 必要性说明
   1. 《我的世界》中, 似乎都需要描边
2. 实现思想：
   1. 如果当前像素是透明像素, 且下一个像素不是透明像素, 则该像素为边缘像素, 将之加深
   2. 如果当前像素不是透明像素, 且下一个像素是透明像素, 则该像素为边缘像素, 将之加深

In [13]:
def is_edge(a, a_1): 
    if (a==0 and a_1 !=0):
        return 'next_is_edge'
    elif (a!=0 and a_1 ==0):
        return 'now_is_edge'
    return False

# 超分辨率函数 

In [14]:
import torch
import torchvision

# 函数定义, 将32*32的风格图像放大到512*512, 原图中的每个像素向右下扩散16*16个像素
def super_resolution_to_512(img):
    img = img.convert('RGB')
    img_torch = torchvision.transforms.ToTensor()(img)
    # print(img_torch.size())
    img_512 = torch.zeros((3, 512, 512))
    # print(img_512.size())
    for i in range(0,img_torch.size()[0],):
        for j in range(0, img_torch.size()[1]):
            for k in range(0, img_torch.size()[2]):
                img_512[i, 16*j:16*(j+1), 16*k:16*(k+1)] = img_torch[i, j, k]
    # print(img_512.size())
    return img_512

In [15]:
from torchvision import utils as vutils

pth = '/home/zxt/Python_area/for_wu_ding_minecraft_modules/datas/1_origin_data/bamboo_large_leaves.png'
img = Image.open(pth)
img = super_resolution_to_512(img)

# 正式运行程序

## 去除图像白色与黑色背景

1. 记录输入与输出路径

In [16]:
org_pics_path = "/home/zxt/Python_area/for_wu_ding_minecraft_modules/datas/1_origin_data/bamboo_large_leaves_512.png"
after_processed_path = "./datas/2_preprocessed_data/preprocessed_bamboo_large_leaves_512.png"

2. 去除图像白色与黑色背景

In [18]:
image = Image.open(org_pics_path)

# Convert the image to RGBA mode
image = image.convert("RGBA")

# Get the pixel data
pixels = image.load()

print(image.width, image.height)

# Iterate over each pixel
for i in range(image.width):
    for j in range(image.height):
        # Get the RGB values of the pixel
        r, g, b, a = pixels[i, j]
        # print(f"r={r}, g={g}, b={b}, a={a} at ({i}, {j})")

        # 检测像素是否为高明度低饱和
        if judge_low_s_high_l(r, g, b):
            # Set the alpha value to 0 (transparent)
            pixels[i, j] = (r, g, b, 0)
        
        #检测像素是否为低明度
        elif judge_low_l(r, g, b):
            # Set the alpha value to 0 (transparent)
            pixels[i, j] = (r, g, b, 0)

        # 检测像素是否接近白色
        elif is_near_white(pixels[i, j]):
            # Set the alpha value to 0 (transparent)
            pixels[i, j] = (r, g, b, 0)
        else:
            continue     
            


# Save the modified image
image.save(after_processed_path)


512 512


## 对图像进行描边操作

In [8]:
image = Image.open(after_processed_path)

# Convert the image to RGBA mode
image = image.convert("RGBA")

# Get the pixel data
pixels = image.load()

print(image.width, image.height)

for i in range(image.width):
    for j in range(image.height):
        # 防止数组越界
        if i < image.width-1:
            # 如果当前像素为透明像素, 右侧的像素不是透明像素, 说明右侧的像素是边缘像素, 则将右侧的像素设置为黑色
            if is_edge(pixels[i, j][3], pixels[i+1, j][3])=="next_is_edge":
                pixels[i+1, j] = (0,0,0,1)
            # 如果当前像素是非透明像素, 右侧的像素是透明像素, 说明当前像素是边缘像素, 则将当前像素设置为黑色
            elif is_edge(pixels[i, j][3], pixels[i+1, j][3])=="now_is_edge":
                pixels[i, j] = (0,0,0,1)
        if j < image.height-1:
            # 如果当前像素为透明像素, 下方的像素不是透明像素, 说明下方的像素是边缘像素, 则将下方的像素设置为黑色
            if is_edge(pixels[i, j][3], pixels[i, j+1][3])=="next_is_edge":
                pixels[i, j+1] = (0,0,0,1)
            # 如果当前像素是非透明像素, 下方的像素是透明像素, 说明当前像素是边缘像素, 则将当前像素设置为黑色
            elif is_edge(pixels[i, j][3], pixels[i, j+1][3])=="now_is_edge":
                pixels[i, j] = (0,0,0,1)


# Save the modified image
after_processed_path = "./datas/2_preprocessed_data/preprocessed_tomatoes_edge.png"
image.save(after_processed_path)

768 1024
